In [93]:
import org.apache.spark.sql.functions._
import org.apache.spark.SparkContext._
import org.apache.spark.sql.types._
import org.apache.spark.mllib.linalg.Vector
import org.apache.spark.sql.functions.{col, udf}

lastException: Throwable = null


In [2]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}

In [3]:
val spark = SparkSession.builder().appName("test").config("spark.master", "yarn").getOrCreate()

spark = org.apache.spark.sql.SparkSession@39287a4


org.apache.spark.sql.SparkSession@39287a4

In [4]:
val train = spark.read
                    .option("delimiter", ",")
                    .option("header", "True")
                    .csv("/labs/slaba05/lab05_train.csv")
val test = spark.read
                    .option("delimiter", ",")
                    .option("header", "True")
                    .csv("/labs/slaba05/lab05_test.csv")

train = [_c0: string, ID: string ... 115 more fields]
test = [_c0: string, ID: string ... 114 more fields]


[_c0: string, ID: string ... 114 more fields]

In [7]:
train.show(1)

+------+------+--------------+------------------+------------------------+--------------------+-----------------+----------------+----------------------+-----------------------+-----------------------+------------------+---------------------+-----------------------+-------------------+--------------+-----------------+--------------------+-----------------------+--------------+------------------+--------------------+---------------+----------------+-----------------------+---------------------------+----------------------+------------------+-----------------+-------------------+-------------------+-------------+-----------------------+-----------------------+-----------------+---------------+-------------------+---------------+---------------+----------------+-------+---------------+-----------------------+-----------------+-----------+-------------------+------------------+-------------+---+-----------------------+--------------+-----------------------+--------------------+------------

In [61]:
var train1 = train.select("ID", "TARGET", "TRANS_CNT_TENDENCY3M", "TURNOVER_DYNAMIC_CC_3M", "TURNOVER_DYNAMIC_CUR_3M",
                            "TURNOVER_DYNAMIC_IL_3M", "TURNOVER_DYNAMIC_PAYM_3M",
                           "SUM_TRAN_SUP_TENDENCY3M", "SUM_TRAN_MED_TENDENCY3M", "SUM_TRAN_CLO_TENDENCY3M", "SUM_TRAN_AUT_TENDENCY3M",
                           "REST_DYNAMIC_SAVE_3M", "REST_DYNAMIC_PAYM_3M", "REST_DYNAMIC_IL_3M", "REST_DYNAMIC_FDEP_3M",
                           "REST_DYNAMIC_CUR_3M", "REST_DYNAMIC_CC_3M",
                           "CNT_TRAN_SUP_TENDENCY3M", "CNT_TRAN_MED_TENDENCY3M", "CNT_TRAN_CLO_TENDENCY3M", "CNT_TRAN_AUT_TENDENCY3M",
                           "CNT_TRAN_ATM_TENDENCY3M")
var test1 = test.select("ID",  "TRANS_CNT_TENDENCY3M", "TURNOVER_DYNAMIC_CC_3M", "TURNOVER_DYNAMIC_CUR_3M",
                        "TURNOVER_DYNAMIC_IL_3M", "TURNOVER_DYNAMIC_PAYM_3M",
                       "SUM_TRAN_SUP_TENDENCY3M", "SUM_TRAN_MED_TENDENCY3M", "SUM_TRAN_CLO_TENDENCY3M", "SUM_TRAN_AUT_TENDENCY3M",
                       "REST_DYNAMIC_SAVE_3M", "REST_DYNAMIC_PAYM_3M", "REST_DYNAMIC_IL_3M", "REST_DYNAMIC_FDEP_3M",
                       "REST_DYNAMIC_CUR_3M", "REST_DYNAMIC_CC_3M",
                       "CNT_TRAN_SUP_TENDENCY3M", "CNT_TRAN_MED_TENDENCY3M", "CNT_TRAN_CLO_TENDENCY3M", "CNT_TRAN_AUT_TENDENCY3M",
                       "CNT_TRAN_ATM_TENDENCY3M")

train1 = [ID: string, TARGET: string ... 20 more fields]
test1 = [ID: string, TRANS_CNT_TENDENCY3M: string ... 19 more fields]


lastException: Throwable = null


[ID: string, TRANS_CNT_TENDENCY3M: string ... 19 more fields]

In [63]:
var feature_list = Array[String]()
for (t <- train1.columns if (t!="ID"))
{ train1 = train1.withColumn(t, col(t).cast(FloatType))
        feature_list +:= t}
val b = feature_list.filter(! _.contains("TARGET"))

feature_list = Array(CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_AUT_TENDENCY3M, CNT_TRAN_CLO_TENDENCY3M, CNT_TRAN_MED_TENDENCY3M, CNT_TRAN_SUP_TENDENCY3M, REST_DYNAMIC_CC_3M, REST_DYNAMIC_CUR_3M, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_3M, REST_DYNAMIC_PAYM_3M, REST_DYNAMIC_SAVE_3M, SUM_TRAN_AUT_TENDENCY3M, SUM_TRAN_CLO_TENDENCY3M, SUM_TRAN_MED_TENDENCY3M, SUM_TRAN_SUP_TENDENCY3M, TURNOVER_DYNAMIC_PAYM_3M, TURNOVER_DYNAMIC_IL_3M, TURNOVER_DYNAMIC_CUR_3M, TURNOVER_DYNAMIC_CC_3M, TRANS_CNT_TENDENCY3M, TARGET)
b = Array(CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_AUT_TENDENCY3M, CNT_TRAN_CLO_TENDENCY3M, CNT_TRAN_MED_TENDENCY3M, CNT_TRAN_SUP_TENDENCY3M, REST_DYNAMIC_CC_3M, REST_DYNAMIC_CUR_3M, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_3M, REST_DYNAMIC_PAYM_3M, REST_DYNAMIC_SAVE_3M, SUM...


Array(CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_AUT_TENDENCY3M, CNT_TRAN_CLO_TENDENCY3M, CNT_TRAN_MED_TENDENCY3M, CNT_TRAN_SUP_TENDENCY3M, REST_DYNAMIC_CC_3M, REST_DYNAMIC_CUR_3M, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_3M, REST_DYNAMIC_PAYM_3M, REST_DYNAMIC_SAVE_3M, SUM...

In [64]:
for (t <- test1.columns if (t!="ID"))
{ test1 = test1.withColumn(t, col(t).cast(FloatType))}

In [65]:
test1 = test1.na.fill(value=0.0)
train1 = train1.na.fill(value=0.0)

test1 = [ID: string, TRANS_CNT_TENDENCY3M: float ... 19 more fields]
train1 = [ID: string, TARGET: float ... 20 more fields]


[ID: string, TARGET: float ... 20 more fields]

In [66]:
val assembler = new VectorAssembler()
                    .setInputCols(b)
                    .setOutputCol("features")

assembler = vecAssembler_cc981d3894f2


vecAssembler_cc981d3894f2

In [67]:
train1 = train1.withColumn("TARGET", col("TARGET").cast(IntegerType))

train1 = [ID: string, TARGET: int ... 20 more fields]


[ID: string, TARGET: int ... 20 more fields]

In [68]:
train1.schema

StructType(StructField(ID,StringType,true), StructField(TARGET,IntegerType,true), StructField(TRANS_CNT_TENDENCY3M,FloatType,false), StructField(TURNOVER_DYNAMIC_CC_3M,FloatType,false), StructField(TURNOVER_DYNAMIC_CUR_3M,FloatType,false), StructField(TURNOVER_DYNAMIC_IL_3M,FloatType,false), StructField(TURNOVER_DYNAMIC_PAYM_3M,FloatType,false), StructField(SUM_TRAN_SUP_TENDENCY3M,FloatType,false), StructField(SUM_TRAN_MED_TENDENCY3M,FloatType,false), StructField(SUM_TRAN_CLO_TENDENCY3M,FloatType,false), StructField(SUM_TRAN_AUT_TENDENCY3M,FloatType,false), StructField(REST_DYNAMIC_SAVE_3M,FloatType,false), StructField(REST_DYNAMIC_PAYM_3M,FloatType,false), StructField(REST_DYNAMIC_IL_3M,FloatType,false), StructField(REST_DYNAMIC_FDEP_3M,F...

In [69]:
val train2=assembler.transform(train1)

train2 = [ID: string, TARGET: int ... 21 more fields]


[ID: string, TARGET: int ... 21 more fields]

In [58]:
val evaluator = new BinaryClassificationEvaluator()
                        .setLabelCol("TARGET")
                        .setRawPredictionCol("probability")
                        .setMetricName("areaUnderROC")

evaluator = binEval_38d9fde76a6e


binEval_38d9fde76a6e

In [70]:
val rf = new RandomForestClassifier()
      .setFeaturesCol("features")
      .setLabelCol("TARGET")
      .setNumTrees(50)
      .setMaxDepth(10)
      .setSubsamplingRate(0.7)

rf = rfc_8ca846d5b0b6


rfc_8ca846d5b0b6

In [71]:
val test2=assembler.transform(test1)

test2 = [ID: string, TRANS_CNT_TENDENCY3M: float ... 20 more fields]


[ID: string, TRANS_CNT_TENDENCY3M: float ... 20 more fields]

In [72]:
val model_rf = rf.fit(train2)

model_rf = RandomForestClassificationModel (uid=rfc_8ca846d5b0b6) with 50 trees


RandomForestClassificationModel (uid=rfc_8ca846d5b0b6) with 50 trees

In [73]:
val predictions_train = model_rf.transform(train2)

predictions_train = [ID: string, TARGET: int ... 24 more fields]


[ID: string, TARGET: int ... 24 more fields]

In [81]:
val predictions_test = model_rf.transform(test2)

predictions_test = [ID: string, TRANS_CNT_TENDENCY3M: float ... 23 more fields]


[ID: string, TRANS_CNT_TENDENCY3M: float ... 23 more fields]

In [100]:
val vecToArray = udf((v: Vector) =>  v.toArray)

vecToArray = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),Some(List(org.apache.spark.mllib.linalg.VectorUDT@f71b0bce)))


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),Some(List(org.apache.spark.mllib.linalg.VectorUDT@f71b0bce)))

In [96]:
predictions_test.printSchema()

root
 |-- ID: string (nullable = true)
 |-- TRANS_CNT_TENDENCY3M: float (nullable = false)
 |-- TURNOVER_DYNAMIC_CC_3M: float (nullable = false)
 |-- TURNOVER_DYNAMIC_CUR_3M: float (nullable = false)
 |-- TURNOVER_DYNAMIC_IL_3M: float (nullable = false)
 |-- TURNOVER_DYNAMIC_PAYM_3M: float (nullable = false)
 |-- SUM_TRAN_SUP_TENDENCY3M: float (nullable = false)
 |-- SUM_TRAN_MED_TENDENCY3M: float (nullable = false)
 |-- SUM_TRAN_CLO_TENDENCY3M: float (nullable = false)
 |-- SUM_TRAN_AUT_TENDENCY3M: float (nullable = false)
 |-- REST_DYNAMIC_SAVE_3M: float (nullable = false)
 |-- REST_DYNAMIC_PAYM_3M: float (nullable = false)
 |-- REST_DYNAMIC_IL_3M: float (nullable = false)
 |-- REST_DYNAMIC_FDEP_3M: float (nullable = false)
 |-- REST_DYNAMIC_CUR_3M: float (nullable = false)
 |-- REST_DYNAMIC_CC_3M: float (nullable = false)
 |-- CNT_TRAN_SUP_TENDENCY3M: float (nullable = false)
 |-- CNT_TRAN_MED_TENDENCY3M: float (nullable = false)
 |-- CNT_TRAN_CLO_TENDENCY3M: float (nullable = false

lastException: Throwable = null


In [ ]:
val to_file = predictions_test.select(col("ID").alias("id"), vecToArray(col("probability")).getItem(1).alias("target")).coalesce(1)

In [91]:
predictions_test.select("probability").show(1, false)

+----------------------------------------+
|probability                             |
+----------------------------------------+
|[0.7980660019990831,0.20193399800091705]|
+----------------------------------------+
only showing top 1 row



In [ ]:
to_file.show()

In [ ]:
import java.io.PrintWriter
new PrintWriter("lab05.csv") { write(to_file); close }